## Load pretrained BERT

In [430]:
# bert_path = "/Users/lukas/Desktop/Projects/MIT/MIT_prosody/precomputed/babylm_runs/20-18-29_text_only/checkpoints/bert"
bert_path = "/Users/lukas/Desktop/Projects/MIT/MIT_prosody/precomputed/babylm_runs/23-00-09_multi/checkpoints/bert"

In [445]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
import torch

# Load pre-trained model tokenizer (vocabulary)
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer = BertTokenizer.from_pretrained(
    "/Users/lukas/Desktop/Projects/MIT/MIT_prosody/src/tokenizer/tokenizer-trained-babylm_100M"
)
model = BertForMaskedLM.from_pretrained(bert_path).eval()

In [464]:
# load pack
import pickle

with open(
    "/Users/lukas/Desktop/Projects/MIT/data/peoples_speech/packs/100M_local_5/sample_pack_1987.pkl",
    "rb",
) as f:
    pack = pickle.load(f)

In [465]:
texts = [p[1] for p in pack]
texts[:3]

["Hi, this is Bill Safer, your host of Hidden Treasures, and welcome to another episode. This is WCCA-TV coming to you from its studios in downtown Worcester. It's visible on cable in Worcester on Channel one-nine-four and worldwide on the Internet at www.wccatv.com. And before I introduce my very special",
 'guest, I would like to do some shout-outs. Cookies are for sale at New Teaks at the Barn in Paxton, owned by Linda Katsopoulos, who has been a guest on this show two times. You can get a double treat by visiting New Teaks at the Barn and buying a',
 "delicious cookie when you leave. So you'll have double prices. I'd like to also say hello to C.C. Lowell and their new location on Four Fifty-Five Pleasant Street not too far from Park Ave in Worcester and they have been in business over a century. They're the second oldest business"]

In [448]:
def _mask_text_tokens(inputs, mask_rate):
    """
    inputs: input_ids, attention_mask, token_type_ids
    """
    labels = inputs.clone()
    inputs_cloned = inputs.clone()

    probability_matrix = torch.full(labels.shape, mask_rate)

    # Determine which tokens to mask
    masked_indices = torch.bernoulli(probability_matrix).bool()

    # Mask tokens
    inputs_cloned[masked_indices] = tokenizer.convert_tokens_to_ids("[MASK]")

    # Replace -100 in labels that we do not want to compute the loss for
    labels[~masked_indices] = -100

    return inputs_cloned, labels

In [454]:
text = """
Former President Donald J. Trump was charged with four counts in connection with his efforts to subvert the will of voters in 2020. “Despite having lost, the defendant was determined to remain in power,” prosecutors wrote."""

text = texts[1]
print(text)

# Tokenize the text
input_ids = tokenizer.encode(text, return_tensors="pt")
print(f"Input ids: {input_ids}")

# mask tokens
input_ids, labels = _mask_text_tokens(input_ids, 0.5)
# input_ids, labels

guest, I would like to do some shout-outs. Cookies are for sale at New Teaks at the Barn in Paxton, owned by Linda Katsopoulos, who has been a guest on this show two times. You can get a double treat by visiting New Teaks at the Barn and buying a
Input ids: tensor([[    2, 10771,    16,    51,  5876,  5808,  5667,  5722,  5867,  8982,
            17, 17461,    18, 14822,  5742,  5707, 11415,  5743,  5983,  8474,
          6132,  5743,  5653, 11568,  5672,  6633, 18812,    16,  9894,  5785,
         13395, 10431,  5968, 12654, 30362,  4300,    16,  5828,  5901,  5931,
            43, 10771,  5691,  5732,  6211,  5958,  6603,    18,  5668,  5765,
          5821,    43,  8829,  7419,  5785, 13658,  5983,  8474,  6132,  5743,
          5653, 11568,  5670, 11836,    43,     3]])


In [455]:
# model = BertForMaskedLM.from_pretrained('bert-base-cased')

In [456]:
outputs = model(input_ids=input_ids, labels=labels)

Bert input ids [0]: tensor([    2,     4,     4,    51,     4,  5808,     4,     4,  5867,     4,
            4, 17461,    18,     4,     4,     4,     4,     4,  5983,     4,
         6132,     4,     4, 11568,  5672,  6633, 18812,     4,     4,     4,
        13395,     4,  5968, 12654,     4,     4,     4,  5828,  5901,     4,
            4,     4,  5691,     4,  6211,     4,  6603,     4,     4,  5765,
         5821,     4,  8829,  7419,  5785, 13658,     4,     4,  6132,     4,
            4, 11568,     4, 11836,     4,     4])
Bert labels [0]: tensor([ -100, 10771,    16,  -100,  5876,  -100,  5667,  5722,  -100,  8982,
           17,  -100,  -100, 14822,  5742,  5707, 11415,  5743,  -100,  8474,
         -100,  5743,  5653,  -100,  -100,  -100,  -100,    16,  9894,  5785,
         -100, 10431,  -100,  -100, 30362,  4300,    16,  -100,  -100,  5931,
           43, 10771,  -100,  5732,  -100,  5958,  -100,    18,  5668,  -100,
         -100,    43,  -100,  -100,  -100,  -100,  598

In [457]:
outputs.loss

tensor(0.0006, grad_fn=<NllLossBackward0>)

In [459]:
pred_tokens = torch.tensor(
    [
        2,
        10771,
        16,
        5700,
        5876,
        5808,
        5667,
        5722,
        5867,
        8982,
        17,
        17461,
        18,
        14822,
        5742,
        5707,
        11415,
        5743,
        5722,
        8474,
        6132,
        5743,
        5653,
        11568,
        5672,
        6633,
        18812,
        16,
        9894,
        5785,
        13395,
        10431,
        5968,
        12654,
        30362,
        4300,
        16,
        5808,
        5901,
        5931,
        43,
        10771,
        5691,
        5732,
        6211,
        5958,
        6603,
        18,
        5668,
        5949,
        5700,
        43,
        8829,
        7419,
        5785,
        13658,
        5983,
        8474,
        6132,
        5743,
        5653,
        11568,
        5670,
        11836,
        43,
        3,
    ]
)

tokenizer.decode(pred_tokens)

'[CLS] guest, we would like to do some shout - outs. cookies are for sale at do teaks at the barn in paxton, owned by linda katsopoulos, like has been a guest on this show two times. you could we a double treat by visiting new teaks at the barn and buying a [SEP]'